In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from eval_utils import *
from video_utils import *
from load_utils import *
from tracking_utils import *

data_path = '../../AICity_data/train/S03/c010/'

In [2]:
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')
gt_notParked = getNotParkedCars(gt_detect)

In [3]:
retinanet_101_detections = {}
with open('./detection_pkls/retinanet_101_detections.pkl','rb') as openFile:
    retinanet_101_detections = pickle.load(openFile)

In [4]:
def remove_overlaps(detections_pd, tolerance=0.9):
    row_to_remove = []
    for detection in detections_pd.get('detection'):
        length, _ = detections_pd.shape
        for i in range(length):
            IoU = detection.IoU(detections_pd.iloc[i]['detection'])
            if IoU > tolerance and IoU < 1:
                row_to_remove.append(i)
                
    row_to_remove = np.unique(np.array(row_to_remove))
    detections_pd = detections_pd.drop(index=row_to_remove)
        
    return detections_pd

In [5]:
def get_detection_dataframe(detections):
    # For detetections in one frame!
    
    bboxes = []
    bsizes = []
    bdetections = []
    tracks = list(range(0, len(detections)))
    
    colours = []
    for i in range(len(detections)):
        colours.append(tuple(np.random.choice(range(256), size=3).astype('int')))
    
    for detection in detections:
        bbox = np.array(detection.getBBox()).astype('int')
        bboxes.append(bbox)

        bsize = int(detection.areaOfRec())
        bsizes.append(bsize)

        bdetections.append(detection)

    detec = {
        'track': tracks,
        'detection': bdetections,
        'bbox': bboxes,
        'size': bsizes,
        'colour': colours,
    }
    detections_pd = pd.DataFrame(detec)
    # detections_pd = detections_pd.sort_values(by=['size'], ascending=False)
    detections_pd = detections_pd.reset_index(drop=True)
    
    return detections_pd

### GT detections

In [6]:
gt_detections_pd = get_detection_dataframe(gt_detect['535'])
gt_detections_pd

,track,detection,bbox,size,colour
0,0,"Frame 535, TL [579.31,420.6], BR [830.83,626.0...","[579, 420, 830, 626]",51662,"(124, 108, 218)"
1,1,"Frame 535, TL [656.55,191.08], BR [809.23,306....","[656, 191, 809, 306]",17694,"(172, 158, 163)"
2,2,"Frame 535, TL [1287.95,149.84], BR [1534.79,26...","[1287, 149, 1534, 269]",29477,"(51, 73, 51)"
3,3,"Frame 535, TL [558.36,94.45], BR [663.49,169.3...","[558, 94, 663, 169]",7870,"(108, 213, 70)"
4,4,"Frame 535, TL [573.7,72.89], BR [661.41,145.05...","[573, 72, 661, 145]",6329,"(245, 183, 113)"
5,5,"Frame 535, TL [913.24,93.02], BR [972.14,144.5...","[913, 93, 972, 144]",3036,"(115, 121, 37)"
6,6,"Frame 535, TL [894.24,95.32], BR [944.92,144.0...","[894, 95, 944, 144]",2467,"(244, 176, 21)"
7,7,"Frame 535, TL [878.4,107.28], BR [927.5,145.72...","[878, 107, 927, 145]",1887,"(22, 1, 224)"
8,8,"Frame 535, TL [1176.01,82.17], BR [1253.12,174...","[1176, 82, 1253, 174]",7087,"(1, 52, 10)"
9,9,"Frame 535, TL [1285.84,363.23], BR [1516.36,54...","[1285, 363, 1516, 546]",42341,"(45, 191, 203)"


In [7]:
det_detections_pd = get_detection_dataframe(retinanet_101_detections['535'])
det_detections_pd = remove_overlaps(det_detections_pd, 0.9)
det_detections_pd

,track,detection,bbox,size,colour
0,0,"Frame 535, TL [1289.42041015625,150.7349548339...","[1289, 150, 1535, 266]",28457,"(70, 24, 207)"
1,1,"Frame 535, TL [588.1416625976562,420.747833251...","[588, 420, 832, 620]",48829,"(40, 94, 194)"
2,2,"Frame 535, TL [1285.02294921875,361.4525451660...","[1285, 361, 1521, 542]",42678,"(149, 47, 125)"
3,3,"Frame 535, TL [661.740234375,192.6010437011718...","[661, 192, 807, 302]",15935,"(172, 105, 40)"
4,4,"Frame 535, TL [563.8505249023438,95.7746124267...","[563, 95, 659, 168]",6932,"(53, 148, 139)"
5,5,"Frame 535, TL [928.1326293945312,78.9821014404...","[928, 78, 1015, 146]",5897,"(131, 144, 194)"
6,6,"Frame 535, TL [887.8241577148438,95.1392211914...","[887, 95, 938, 142]",2364,"(40, 184, 88)"
7,7,"Frame 535, TL [876.5736694335938,99.0433807373...","[876, 99, 917, 142]",1793,"(18, 32, 135)"
8,8,"Frame 535, TL [910.677001953125,92.69955444335...","[910, 92, 945, 141]",1706,"(22, 34, 252)"
9,9,"Frame 535, TL [559.1068725585938,72.7742919921...","[559, 72, 659, 132]",6019,"(255, 143, 132)"


In [10]:
total_frames = 2141
initial_frame = 535

acc = create_accumulator()

for frame in range(initial_frame, total_frames):
    sframe = str(frame)

    gt_detections_pd = get_detection_dataframe(gt_detect[sframe])
    det_detections_pd = get_detection_dataframe(retinanet_101_detections[sframe])
    
    gt_object_ids = []
    for track in gt_detections_pd['track']:
        gt_object_ids.append(track)

    det_object_ids = []
    for track in det_detections_pd['track']:
        det_object_ids.append(track)

    IoU_ratings = []
    for track_gt in gt_object_ids:
        det_IoUs = []
        for track_det in det_object_ids:
            IoU_inverted = 1-(gt_detections_pd[gt_detections_pd['track'] == track_gt].detection.item()).IoU(det_detections_pd[det_detections_pd['track'] == track_det].detection.item())
            det_IoUs.append(IoU_inverted)
        IoU_ratings.append(det_IoUs)

    acc = update_accumulator(acc, gt_object_ids, det_object_ids, IoU_ratings)

display_metrics(acc)


KeyboardInterrupt: 